# Operations on Streaming Dataframes/Datasets Demo

### Demo

In [1]:
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType

In [2]:
findspark.init('/opt/spark-3.5.0/')

spark = SparkSession\
    .builder\
    .appName("UserInteractionAnalyzer")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/25 20:51:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/25 20:51:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
staging_dir = 'monitoring_data'

In [4]:
# Read all the csv files written atomically in a directory
# The schema is as follows:
# userA, userB, timestamp, interaction
user_schema = StructType()\
    .add("userA", "integer")\
    .add("userB", "integer")\
    .add("timestamp", 'timestamp')\
    .add("interaction", "string")

In [5]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
activity = spark\
    .readStream\
    .option("sep", ",")\
    .schema(user_schema)\
    .csv(staging_dir + "/*.csv")

In [6]:
# user_ab = activity.select(['userA', 'userB']).where('interaction = "MT"')
user_b = activity\
    .select("userB")\
    .where("interaction = \"MT\"")

In [ ]:
# query = user_b \
#     .writeStream \
#     .trigger(processingTime='10 seconds') \
#     .format("parquet") \
#     .option("checkpointLocation", "applicationHistory") \
#     .option("path",staging_dir + "/out") \
#     .start()

query2 = user_b\
    .writeStream\
    .trigger(processingTime='10 seconds')\
    .format("console")\
    .start()

query2.awaitTermination()
# spark.sql("select * from aggregates").show()   # interactively query in-memory table

24/03/25 20:51:15 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/v4/r34w0mtd5xzfzyxp6hnbp8y00000gn/T/temporary-b9c0ecdd-a682-4517-910d-79a93c3d2783. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/25 20:51:15 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+
| userB|
+------+
|   383|
|  1454|
| 38034|
| 29600|
|    88|
|  4022|
|261475|
| 33476|
|146171|
| 59195|
| 20802|
| 14376|
|    88|
| 50901|
| 31957|
| 16783|
|    88|
|406395|
| 68285|
| 30978|
+------+
only showing top 20 rows

